In [6]:
import os
import re
import glob
import rasterio
import subprocess
import numpy as np
import pandas as pd
from osgeo import ogr, gdal, osr
import geopandas as gpd
from rasterio import mask
from rasterio.plot import show
import matplotlib.pyplot as plt
from rasterio.merge import merge
from geopy.distance import geodesic
from rasterio.windows import Window
from shapely.geometry import mapping
from rasterio.enums import Resampling

In [7]:
# Testing how to get coordinates from pixels

def get_lat_lon_from_pixel(geo_transform, col, row):
    lon = geo_transform[0] + col * geo_transform[1] + row * geo_transform[2]
    lat = geo_transform[3] + col * geo_transform[4] + row * geo_transform[5]
    return lat, lon

def pixel_coordinates_in_other_image(src_geo_transform, dst_geo_transform, col, row):
    # Convert pixel coordinates from source image to geographic coordinates
    src_lat, src_lon = get_lat_lon_from_pixel(src_geo_transform, col, row)
    
    # Convert geographic coordinates to pixel coordinates in the destination image
    dst_col = int((src_lon - dst_geo_transform[0]) / dst_geo_transform[1])
    dst_row = int((src_lat - dst_geo_transform[3]) / dst_geo_transform[5])
    
    return dst_col, dst_row

def get_day_from_filename(file_path):
    # Use regular expression to extract the day from the filename
    match = re.search(r'doy(\d{3})', file_path)
    if match:
        return int(match.group(1))
    else:
        return 0  # Return 0 if day information is not found

def correlate_pixel_coordinates(src_tif_path, dst_tif_path):
    # Open the source GeoTIFF file
    src_dataset = gdal.Open(src_tif_path)
    src_geo_transform = src_dataset.GetGeoTransform()

    # Open the destination GeoTIFF file
    dst_dataset = gdal.Open(dst_tif_path)
    dst_geo_transform = dst_dataset.GetGeoTransform()

    # Loop through each pixel in the source image
    for row in range(src_dataset.RasterYSize):
        for col in range(src_dataset.RasterXSize):
            # Get the pixel value
            pixel_value = src_dataset.ReadAsArray(col, row, 1, 1)

            # Get the latitude and longitude of the pixel in the source image
            src_lat, src_lon = get_lat_lon_from_pixel(src_geo_transform, col, row)

            # Convert pixel coordinates to the destination image
            dst_col, dst_row = pixel_coordinates_in_other_image(src_geo_transform, dst_geo_transform, col, row)

            # Get the latitude and longitude of the pixel in the destination image
            dst_lat, dst_lon = get_lat_lon_from_pixel(dst_geo_transform, dst_col, dst_row)

            print(f"Source Pixel: ({col}, {row}), Value: {pixel_value}, Source Lat/Lon: ({src_lat}, {src_lon})")
            print(f"Destination Pixel: ({dst_col}, {dst_row}), Destination Lat/Lon: ({dst_lat}, {dst_lon})")

    # Close the datasets
    src_dataset = None
    dst_dataset = None

    
# Replace 'input.tif' and 'vegetation.tif' with the paths to your GeoTIFF files
input_tif = f'input.tif'
gfc_tif = f'second_input.tif'
correlate_pixel_coordinates(input_tif, gfc_tif)


ERROR 4: input.tif: No such file or directory


AttributeError: 'NoneType' object has no attribute 'GetGeoTransform'

In [1]:
# Should later be replaced with a hashmap using the name of each ecoregion as a key
# and the starting coordinate for analysis as the value
eco_starting_coord = (-12.46796,-54.79239)

In [3]:
# Script for analyzing each ecoregion
def latlon_to_pixel(lat, lon, geotransform):
    """
    Convert latitude and longitude to pixel coordinates in a GeoTIFF file.

    Parameters:
    - lat: Latitude
    - lon: Longitude
    - geotransform: GeoTransform information from the GeoTIFF file

    Returns:
    - x_pixel: Pixel x-coordinate
    - y_pixel: Pixel y-coordinate
    """

    # Extracting GeoTransform parameters
    x_geo, pixel_width, _, y_geo, _, pixel_height = geotransform

    # Calculating pixel coordinates
    x_pixel = int((lon - x_geo) / pixel_width)
    y_pixel = int((lat - y_geo) / pixel_height)

    return x_pixel, y_pixel

def get_geotransform_from_file(tif_path):
    """
    Get GeoTransform information from a GeoTIFF file.

    Parameters:
    - tif_path: Path to the GeoTIFF file

    Returns:
    - geotransform: GeoTransform information as a tuple
    """
    dataset = gdal.Open(tif_path)
    geotransform = dataset.GetGeoTransform()
    dataset = None  # Close the dataset
    return geotransform

def merge_tiles(year):
    # Input directory containing the TIFF files
    input_directory = f'/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/GFC/GFC_Tiles/20{year}_GFC_tiles'

    # Get a list of all TIFF files in the directory
    input_files = glob.glob(os.path.join(input_directory, '*.tif'))
    
    # print(f"Input directory: {input_directory}")
    # print(f"Input files: {input_files}")

    # Check if there are any TIFF files
    if not input_files:
        print("No TIFF files found in the specified directory.")
        return

    # Read all input files
    src_files_to_mosaic = [rasterio.open(file) for file in input_files]

    # Merge files
    mosaic, out_trans = merge(src_files_to_mosaic, resampling=Resampling.nearest)

    # Output file
    output_file = "/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/QGIS/merged_output.tif"

    # Update metadata of the output file
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({"driver": "GTiff", "height": mosaic.shape[1], "width": mosaic.shape[2], "transform": out_trans, "dtype": src_files_to_mosaic[0].dtypes[0]})

    # Write the mosaic to the output file
    with rasterio.open(output_file, "w", **out_meta) as dest:
        dest.write(mosaic)
        
def get_day_from_filename(file_path):
    # Use regular expression to extract the day from the filename
    match = re.search(r'doy2016(\d{3})', file_path)
    if match:
        return int(match.group(1))
    else:
        return 0  # Return 0 if day information is not found



PRODES = '/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/PRODES/prodes_amazonia_raster_2000_2022_v20231109/prodes_amazonia_raster_2000_2022_v20231109.tif'
ecoregions_dir = '/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/Ecoregions/Brazilian_Amazon_Ecoregions'
APPEARS_dir = '/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/APPEARS/EVI/2015-2023-EVI'
merged_output_dir = '/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/QGIS'
GFC_directory = '/Users/davidcastrejon/Documents/Amazon_Rainforest/Data/GFC/GFC_Tiles'

# List of ecoregion .shp filenames
ecoregions = [filename for filename in os.listdir(ecoregions_dir) if filename.endswith('.shp')]

for ecoregion in ecoregions:
    
    # Create a 31 x 3 grid of subplots
    fig, axes = plt.subplots(nrows=7, ncols=3, figsize=(15, 30))
    # legend_labels = ['Non-deforested', 'Deforested']
    
    # Path to ecoregion shapefile
    shp_path = os.path.join(ecoregions_dir, ecoregion)
    
    # Analyze 2015-2023
    for year in range(16,23):
        !rm *.tif
        !rm *.xml
        
        # Create prodes raster 
        gdal_command_1 = f'gdal_calc.py -A {PRODES} --outfile=out.tif --calc="A=={year}" --NoDataValue=0'
        gdal_command_2 = f'gdalwarp -tr 0.0002689996094039614474 -0.0002690007898141364893 -cutline {shp_path} -crop_to_cutline -dstnodata 0 -of GTiff out.tif prodes_{ecoregion}_20{year}.tif'
        subprocess.run(gdal_command_1, shell=True)
        subprocess.run(gdal_command_2, shell=True)
        
        # Create GFC raster
        merge_tiles(year)
        gdal_command_3 = f'gdalwarp -tr 0.0002689996094039614474 -0.0002690007898141364893 -t_srs EPSG:4674 -cutline {shp_path} -crop_to_cutline -dstnodata 0 -of GTiff merged_output.tif gfc_{ecoregion}_20{year}.tif'
        subprocess.run(gdal_command_3, shell=True)
        
        # Prodes + gfc data will be analyzed by comparing both prodes and gfc together during iterations
        
        # Create ordered list of APPEARS tifs
        pattern = f'MOD13Q1.061__250m_16_days_EVI_doy20{year}*.tif'
        tifs = glob.glob(f'{APPEARS_dir}/{pattern}')
        
        # Sort the file paths based on the day of the year
        appears_tifs = sorted(tifs, key=get_day_from_filename)
        # print(appears_tifs)
        
        # Analysis for each .tif of each year
        for snapshot in appears_tifs:
            # Creates appears.tif with same crs and spatial resolution as prodes data
            gdal_command_4 = f'gdalwarp -tr 0.0002689996094039614474 -0.0002690007898141364893 -t_srs EPSG:4674 -dstnodata 0 -of GTiff {snapshot} appears.tif'
            subprocess.run(gdal_command_4, shell=True)
            
            # Could also use gdal to open raster instead
            with rasterio.open('appears.tif') as apps:
                appears_data = apps.read(1)

            geotransform = get_geotransform_from_file('appears.tif')
            x_pixel, y_pixel = latlon_to_pixel(eco_starting_coord[0], eco_starting_coord[1], geotransform)
            
            # If you want to iterate over the entire ecoregion instead, you can find the coordinate of
            # the upper left corner of either the prodes or gfc .tif files generated above then find
            # which pixel within appears.tif corresponds to that coordinate. Using the size of either
            # prodes or gfc will allow you to then iterate over the entire region since all pixels will be lined up

            '''
            for i in range(x_pixel, x_pixel + 100):
                for j in range(y_pixel, y_pixel + 100):
                    break
                break
            '''
            # !rm appears.tif
            break
        # Breaks year loop
        break
    plt.show()
    # Breaks ecoregion loop
    break
    
# !rm *.tif
plt.close()

NameError: name 'os' is not defined